![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Introduction to `KNN Join` for WherobotsDB

In this notebook we will demostrate how to perform k-Nearest Neighbors (kNN) joins in WherobotsDB.


A geospatial k-Nearest Neighbors (kNN) join is a specialized form of the kNN join that specifically deals with geospatial data. This method involves identifying the k-nearest neighbors for a given spatial point or region based on geographic proximity, typically using spatial coordinates and a suitable distance metric like Euclidean or great-circle distance.

**Approximate kNN Join**

The Z-order based approximate algorithm leverages the properties of Z-order (or Morton order) encoding to efficiently process k-nearest neighbors (kNN) joins in spatial databases. This method maps multidimensional data to one dimension while preserving locality to a certain extent. Here, we outline the algorithm and discuss its application and efficiency in spatial join operations.
    

**Exact kNN Join**

The method uses quad-tree partitioning strategy as a start point. It partitions the dataset $R$ into balanced partitions using the quad-tree strategy, preserving spatial locality. The method then builds an R-tree over a random sample of another dataset $S$ and uses distance bounds to ensure efficient local kNN joins. By calculating distance bounds and using circle range queries, the method ensures that the subsets $S_i$, containing all necessary points for accurate kNN results. The final union of local join results provides the complete kNN join result for the datasets $R$ and $S$.


## Initial Configuration

In [ ]:
from sedona.spark import *

# spark.conf.set("sedona.join.optimizationmode", "all")
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)


# Define Inputs
 - Queries: This table contains the objects for which you want to find the nearest neighbors.
 - Objects: This table contains the objects that are potential neighbors to the objects in the Queries table.

### Queries Table

This table contains the objects for which you want to find the nearest neighbors.

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id, col

# load data
queries_df = sedona.table("wherobots_pro_data.nyc_taxi.yellow_2009_2010").select(col("pickup_location").alias("geometry"))
queries_df = queries_df.withColumn("id", monotonically_increasing_id())

queries_df = queries_df.sample(0.001)
queries_df = queries_df.repartition(100)

queries_df.createOrReplaceTempView("queries")

print(queries_df.rdd.getNumPartitions())
print(queries_df.count())


### Objects Table

This table contains the objects that are potential neighbors to the objects in the Queries table.


In [ ]:
# Load objects table
objects_df = sedona.table("wherobots_open_data.overture_2024_02_15.transportation_connector")

objects_df = objects_df.sample(0.001)
objects_df = objects_df.repartition(200)

objects_df.createOrReplaceTempView("objects")

print(objects_df.rdd.getNumPartitions())
print(objects_df.count())


## Approximate and Exaxct KNN Join (AKNN)

Approximate K-Nearest Neighbors (KNN) Join is a method used to find the nearest neighbors of objects in a high-dimensional space more efficiently than the exact KNN Join. 

The spatial SQL below demonstrates the new SQL syntax for performing approximate KNN on the Wherobots platform.

In [ ]:
# make sure this config is set before executing the knn join
sedona.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

aknn_join_df = sedona.sql("""
SELECT
    QUERIES.GEOMETRY AS QUERIES_GEOM,
    QUERIES.ID AS QID,
    OBJECTS.GEOMETRY AS OBJECTS_GEOM,
    ST_DISTANCESPHERE(QUERIES.GEOMETRY, OBJECTS.GEOMETRY) AS DISTANCE,
    ST_MakeLine(QUERIES.GEOMETRY, OBJECTS.GEOMETRY) AS LINE
FROM QUERIES
JOIN OBJECTS ON ST_AKNN(QUERIES.GEOMETRY, OBJECTS.GEOMETRY, 4, FALSE)
""")


In [ ]:
%%time

# cache for further queries and visualization
aknn_join_df.cache()

total_count = aknn_join_df.count()
print(total_count)


In [ ]:
# Number of unique QIDs needed
N = 8

# Select N unique QID rows
unique_qid_df = aknn_join_df.dropDuplicates(["QID"]).limit(N)

# Perform an inner join to get all rows from join_df that have QIDs in unique_qid_df
related_rows_df = aknn_join_df.join(unique_qid_df, on="QID", how="inner").select(aknn_join_df["*"])

unique_qid_df.cache()
related_rows_df.cache()

related_rows_df.count()

In [ ]:
# create map for the results
map_view = SedonaKepler.create_map(unique_qid_df.select('QUERIES_GEOM'), name="TAXI")
SedonaKepler.add_df(map_view, df=related_rows_df.select('OBJECTS_GEOM', 'DISTANCE').withColumnRenamed("OBJECTS_GEOM", "geometry"), name="TRANSPORTATION HUB")
map_view


In [ ]:
# create map for the results
map_view = SedonaKepler.create_map(unique_qid_df.select('QUERIES_GEOM'), name="TAXI")
SedonaKepler.add_df(map_view, df=related_rows_df.select('OBJECTS_GEOM', 'DISTANCE').withColumnRenamed("OBJECTS_GEOM", "geometry"), name="TRANSPORTATION HUB")
SedonaKepler.add_df(map_view, df=related_rows_df.select('LINE', 'DISTANCE').withColumnRenamed("LINE", "geometry"), name="KNN LINES")
map_view


In [ ]:
# join_df.show()

# Number of unique QIDs needed
N = 100

# Select N unique QID rows
unique_qid_df = aknn_join_df.dropDuplicates(["QID"]).limit(N)

# Perform an inner join to get all rows from join_df that have QIDs in unique_qid_df
related_rows_df = aknn_join_df.join(unique_qid_df, on="QID", how="inner").select(aknn_join_df["*"])

unique_qid_df.cache()
related_rows_df.cache()

# create map for the results
map_view_more = SedonaKepler.create_map(unique_qid_df.select('QUERIES_GEOM'), name="TAXI")
SedonaKepler.add_df(map_view_more, df=related_rows_df.select('OBJECTS_GEOM', 'DISTANCE').withColumnRenamed("OBJECTS_GEOM", "geometry"), name="TRANSPORTATION HUB")
map_view_more
